In [2]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [3]:
!jupyter nbextension enable --py --sys-prefix widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [4]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Kembe,CF,2021-02-05 17:36:37,4.6228,21.8865,81.72,32,21,0.94
1,1,Hilo,US,2021-02-05 17:36:37,19.7297,-155.0900,63.00,67,75,8.05
2,2,Geraldton,AU,2021-02-05 17:32:59,-28.7667,114.6000,64.40,82,90,16.11
3,3,Punta Arenas,CL,2021-02-05 17:36:37,-53.1500,-70.9167,57.20,47,0,26.46
4,4,Bredasdorp,ZA,2021-02-05 17:36:38,-34.5322,20.0403,71.60,73,1,10.36


In [5]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [6]:

# High Temp heatmap
# Get Lat-lon
locations = city_data_df[["Lat","Lng"]]
# Get high temp
max_temp = city_data_df ["Max Temp"]
# Assign fig variable
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign heatmap variable
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp,0) for temp in max_temp], dissipating=False,
                                max_intensity=300,point_radius=4)
# Add heatmap layer
fig.add_layer(heat_layer)
# Call figure
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig


Figure(layout=FigureLayout(height='420px'))

In [8]:
# % Cloudiness heatmap
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300,
                                point_radius=4)
fig.add_layer(heat_layer)
# Call figure
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# Wind speed heatmap
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300,
                                point_radius=4)
fig.add_layer(heat_layer)
# Call figure
fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [11]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Kembe,CF,2021-02-05 17:36:37,4.6228,21.8865,81.72,32,21,0.94
9,9,Rikitea,PF,2021-02-05 17:36:38,-23.1203,-134.9692,78.67,69,27,10.65
11,11,Karratha,AU,2021-02-05 17:33:18,-20.7377,116.8463,79.99,79,12,8.57
12,12,Montego Bay,JM,2021-02-05 17:33:34,18.4712,-77.9188,84.20,74,20,13.80
17,17,Butaritari,KI,2021-02-05 17:36:39,3.0707,172.7902,80.82,83,100,13.78
19,19,Castro,BR,2021-02-05 17:36:40,-24.7911,-50.0119,75.51,52,0,6.78
20,20,Sambava,MG,2021-02-05 17:33:06,-14.2667,50.1667,79.02,80,14,3.58
22,22,Tondano,ID,2021-02-05 17:36:40,1.3038,124.9112,75.20,94,40,2.30
29,29,Tanete,ID,2021-02-05 17:36:41,-3.9443,119.7848,76.37,90,98,4.14
49,49,Rodrigues Alves,BR,2021-02-05 17:36:43,-7.7380,-72.6509,84.20,79,75,3.44


In [12]:
preferred_cities_df.count()

City_ID       161
City          161
Country       159
Date          161
Lat           161
Lng           161
Max Temp      161
Humidity      161
Cloudiness    161
Wind Speed    161
dtype: int64

In [13]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Kembe,CF,81.72,4.6228,21.8865,
9,Rikitea,PF,78.67,-23.1203,-134.9692,
11,Karratha,AU,79.99,-20.7377,116.8463,
12,Montego Bay,JM,84.20,18.4712,-77.9188,
17,Butaritari,KI,80.82,3.0707,172.7902,
19,Castro,BR,75.51,-24.7911,-50.0119,
20,Sambava,MG,79.02,-14.2667,50.1667,
22,Tondano,ID,75.20,1.3038,124.9112,
29,Tanete,ID,76.37,-3.9443,119.7848,
49,Rodrigues Alves,BR,84.20,-7.7380,-72.6509,


In [14]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [15]:
# Set the parameters to search for a hotel in Paris.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key,
    "location": "48.8566, 2.3522"}
# Use base URL to search for hotels in Paris.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# Make request and get the JSON data from the search.
hotels = requests.get(base_url, params=params).json()

hotels

{'html_attributions': [],
 'next_page_token': 'ATtYBwLvFL48moVr_ybvfxYvhIJqwiAfbzEI1lWJbADiVSkm0CxtrXCPPsDkoprHonj2e4tDNH2cmOS9P8SroAmOsbYRMXO3UinrrZhKvRuMx4MIvd1deF_yRM53Vc2JPw1ISDSENQAdjOtaCLbBE8WWwWXjDpyAXlVWWFXXmkhUgH7TNp00TvJ9vXo9jfie3eVhmqedCGOapK6zpRL2SQPFjgs_vW_kyilOblO2fgY4xeRNfgItaTwnVf_a9TJDC5c1WvH7TH_049pe_D5ALAj-LB2Ic1tGwHbWiKj0reLwTVcb5ufxY4nE_vNWvW-d_xn-UURwmJZ3rtW9f5Ja1RM0YR5R7r3TV3b4T5gCnZxcVLYvhfw5t5vDWDHrLDPKEr0hKG_XxpFL8pW5geR0FTsxmJuilS_WPDF64gRreu7f6FTECwKeUf2lBTA',
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 48.8581126, 'lng': 2.3529277},
    'viewport': {'northeast': {'lat': 48.8594393802915,
      'lng': 2.354352980291503},
     'southwest': {'lat': 48.8567414197085, 'lng': 2.351655019708499}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
   'name': 'Hôtel Duo',
   'opening_hours': {'open_now': True},
   'photos': [{'height': 3840,
     'html_attributions': ['<a href="https://m

In [16]:
len(hotels["results"])

20

In [26]:
# Iterate through the DataFrame.

# Iterate through DF
for index, row in hotel_df.iterrows():
    # Get the lat/lon
    lat = row["Lat"]
    lng = row["Lng"]

    # Add lat/lon location key for params dictionary
    params["location"] = f"{lat},{lng}"

    # Use search term: "lodging" and our lat/lon
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get JSON data from search
    hotels = requests.get(base_url, params=params).json()
    # Grab first hotel from results and store name
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... Skipping")

Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping


In [28]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Kembe,CF,81.72,4.6228,21.8865,
9,Rikitea,PF,78.67,-23.1203,-134.9692,Pension Maro'i
11,Karratha,AU,79.99,-20.7377,116.8463,ibis Styles Karratha
12,Montego Bay,JM,84.20,18.4712,-77.9188,Sunscape Splash Montego Bay
17,Butaritari,KI,80.82,3.0707,172.7902,Isles Sunset Lodge
19,Castro,BR,75.51,-24.7911,-50.0119,CHACARA BAILLY
20,Sambava,MG,79.02,-14.2667,50.1667,Melrose
22,Tondano,ID,75.20,1.3038,124.9112,Villa Bella
29,Tanete,ID,76.37,-3.9443,119.7848,Wisma Tri Multi
49,Rodrigues Alves,BR,84.20,-7.7380,-72.6509,Pousada Aconchego


In [30]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig


Figure(layout=FigureLayout(height='420px'))

In [35]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
# add markers
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [ ]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [ ]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig